---
#### 📌 Project : 하루시작 지하철 혼잡도 머신러닝 통합버전 
#### 📌 Description : feature table 통합, 모든 호선 예측 
#### 📌 Author : Forrest Dpark
#### 📌 Date : 2024.06.13+ 
#### 📌 Detail : 
#### 📌 Updates: 
    🟦 2024.06.14 pdg: feature table 통합, 모든 호선 예측 
        ✅ 함수화  
        ✅ 주석   
    - 2024.06.26 pdg : 통합 머신러닝 모델 서버 적용 
        *  restart coding 
        *

---

In [1]:
#### Jupyter Basic Setting ####
import pandas as pd,numpy as np, os,sys # np,pd,os,sys
import matplotlib.pyplot as plt,seaborn as sns  # 시각화
import warnings; warnings.filterwarnings('ignore')  # 경고 무시
# 다른 directory Module 불러오기위한 Setting 
parent_dir = os.path.dirname(os.getcwd() )
sys.path.append(parent_dir)
from Module.Functions import *
Service.plotSetting("seaborn-v0_8")
## Data dictionary save 
# np.save("../Data/DataDictionary/subway_dict_test.npy",subway_dict_22_23) ## 99.8Mb
# subway_dict_22_23 =np.load("../Data/DataDictionary/subway_dict_test.npy",allow_pickle=True).tolist()
# subway_dict_22_23.keys()


___ 🟡 PLOTSETTING. 
______ 📌 matplotlibn plot 한글화 Setting
___## OS platform 한글 세팅완료 ## ___


---
###  Station Info파일을 불러와서 dictionary 화함
    * 위도 경도 merge 

In [2]:

def StationInfo_dataFetch_toDict(parent_dir):
    ##머신러닝을 위해 필요한것 = feature table
    ## feature table 을 만들기 위해 필요한것 -> 지하철 역정보(역사코드,역사이름, 호선, 배차정보) 
    # 역사 정보 정제파일 Fetch
    # 년도별로 역사정보(StationInfo_) 가 Data 폴더 내에 있다. 
    # 이폴더를 list dir 함수를 사용해 읽어들여서 각년도별 역사정보를 dicitionary 로 변수저장한다. 
    StationInfo_dict = {}
    print(Service.colored_text("Parent Directory : "+parent_dir,'green'))
    for i in os.listdir(parent_dir):
        # print(i)
        file_order = 0 # file 순서 
        if i =='Data': # Data 폴더 이면 실행 
            print(Service.colored_text("  |-Data 폴더에서 역사 정보를 가진 StationInfo  csv 파일을 검색합니다.",'yellow'))
            for idx, filename in enumerate(os.listdir(os.path.join(parent_dir,i))):
                # print(" |-",j)
                if 'StationInfo' in filename: # Station info 만 추출
                    file_order += 1
                    print(f" {file_order}. -> ",filename)
                    key_name = filename.split(".csv")[0]
                    # print(key_name)
                    StationInfo_dict[f'{key_name}'] = pd.read_csv(os.path.join(parent_dir,i,filename))
    print(Service.colored_text("  >> 📌 Data folder 에 있는 년도별 Station Info 를  StationInfo_dict 에 저장합니다.", 'blue'))
    print("[[🔹🔹🔹 StationInfo keys 값 정보🔹🔹🔹]]")
    for i in StationInfo_dict.keys():
        print("  |-",Service.colored_text(i,'cyan'))
    
    print(Service.colored_text("  >> 📌 각역에서의 위도경도 데이터를 저장하고 배차시간표를 병합하여 지하철 배차시간데이터폴더에 저장합니다.", 'blue'))
    for key in StationInfo_dict.keys():
        ## 역정보 데이터의  이름 (ex. StationInfo_18) 
        data_name = key
        print(Service.colored_text(f"✅ {data_name} :역사정보 \n",'yellow'),StationInfo_dict[data_name].head(3))
        latlng = pd.read_csv("../Data/seoul_subway_latlon_zenzen.csv")
        latlng.rename( {'고유역번호(외부역코드)':'역사코드'}, inplace=True, axis=1)
        latlng.drop(['역명','호선','환승역수'], axis=1,inplace=True)
        print(Service.colored_text(f"✅ {data_name} : 역사별 위도경도 : \n",'yellow'),latlng.head(3))

        ## 위도 경도 데이터 merge 
        StationInfo_dict[data_name] = pd.merge(StationInfo_dict[data_name],latlng, on='역사코드',how='inner' )
        print(Service.colored_text('✅ Merged data check(tail)','yellow'))
        print(StationInfo_dict[data_name].tail(3))

        # 호선별 배차시간데이터  저장 
        savefilename_indexName = data_name.split('StationInfo')[-1]
        line_배치_dict ={}
        for i in range(1,9):
            try:
                line_배치_dict[f'{i}호선'] = pd.read_csv(f'../Data/지하철배차시간데이터/StationInfo{savefilename_indexName}_{i}_호선배차.csv')
                line_배치_dict[f'{i}호선'] = \
                    Service.table_merge_subwayInfo_dispatch(
                        StationInfo_dict[data_name],
                        line_배치_dict[f'{i}호선'],
                        histPlot=False
                        )
            except: 
                print(f"{key}데이터의{i}호선에 문제 가있습니다")
                continue
        # line_배치_dict['7호선'].tail()
    # return StationInfo_dict ## dictionary 반환 
StationInfo_dict= StationInfo_dataFetch_toDict(parent_dir)


Parent Directory : /Users/forrestdpark/Desktop/PDG/Python_/Project/HaruSijack/DataAnalysis
  |-Data 폴더에서 역사 정보를 가진 StationInfo  csv 파일을 검색합니다.
 1. ->  StationInfo_18.csv
 2. ->  StationInfo_19.csv
 3. ->  StationInfo_22.csv
 4. ->  StationInfo_23_2.csv
 5. ->  StationInfo_23_0.csv
 6. ->  StationInfo_21.csv
 7. ->  StationInfo_20.csv
 8. ->  StationInfo_23_1.csv
 9. ->  StationInfo.csv
  >> 📌 Data folder 에 있는 년도별 Station Info 를  StationInfo_dict 에 저장합니다.
[[🔹🔹🔹 StationInfo keys 값 정보🔹🔹🔹]]
  |- StationInfo_18
  |- StationInfo_19
  |- StationInfo_22
  |- StationInfo_23_2
  |- StationInfo_23_0
  |- StationInfo_21
  |- StationInfo_20
  |- StationInfo_23_1
  |- StationInfo
  >> 📌 각역에서의 위도경도 데이터를 저장하고 배차시간표를 병합하여 지하철 배차시간데이터폴더에 저장합니다.
✅ StationInfo_18 :역사정보 
    역사코드  역이름  호선  환승역수
0   150  서울역   1     2
1   426  서울역   4     2
2   151   시청   1     2
✅ StationInfo_18 : 역사별 위도경도 : 
    역사코드   latitude   longitude
0   150  37.554679  126.970607
1   426  37.554679  126.970607
2   151  37.565345  1

---
### 머신러닝을 위한 feature table 생성후 저장 

In [3]:

def mlTableGen(subway_dict, save_mlTable_dict_fileName):
    """
            # 📌 Description : 머신러닝을 위한 Feature table 생성, 날짜별 승하차정보와 배차정보, 날짜적 특징을 보유한 테이블 생성
            # 📌 Date : 2024.06.14
            # 📌 Author : pdg
            # 📌 Detail:
                03-2.머신러닝 통합.ipynb 에서 활용되는 함수 
                승하차 데이터 dictionary에서 승하차 인원 데이터를 추출하고 년도별 호선별 배차시간표데이터와 조합하여
                머신러닝이 돌아갈수잇는 Feature table 을 생성함.
                🔸 Returns: -> Save files 

            # 📌 Update:

    """
    import pandas as pd, numpy as np, os
    Service.Explaination('ml_Table_Generator','머신러닝을 위한 Feature table 생성, 날짜별 승하차정보와 배차정보, 날짜적 특징을 보유한 테이블 생성')
    # 전체학습데이터 생성 
    # 기본 승하차데이터의 승하차 칼럼들을 다시 정제 해야함. 
    mlTable_dict = {}
    error_table = {}
    for i,key in enumerate(subway_dict.keys()):
        print(Service.colored_text(f'🔹 {i} key : {key}','cyan'))
        mlTable_dict[key]= Service.data_preprocessing_toAnalysis(subway_dict,key)

    print(Service.colored_text("########### 정제된 데이터에서 Feature Engineering 을 시작합니다 #################",'yellow'))

    for i,key in enumerate(mlTable_dict.keys()):
        print(Service.colored_text(f'🔹 {i} key : {key}','cyan'))
        # print(Service.colored_text(f">> 📌{key}data 의 수송일자 칼럼에서 요일(int)을 추출한뒤 4번째 칼럼으로 이동",'yellow'))
        print(Service.colored_text(f'🔹 mlTable 에 날짜를 정제하여 날짜관련 feature 를 생성합니다. ','cyan'))
        mlTable_dict[key]= Service.dayToIntConvert(mlTable_dict[key], "수송일자")
        mlTable_dict[key] = Service.reorder_columns(mlTable_dict[key],col_name="요일",target_idx=4)
        ##날짜에서 년도,월, 주차
        mlTable_dict[key] = Service.date_Divid_Add_YMW_cols(mlTable_dict[key] ,'수송일자')
        ## 날짜에서 공휴일 데이터 추가
        mlTable_dict[key]= Service.holidaysToIntConvert(mlTable_dict[key],DateColName='수송일자')
        ## 수송일자 , 연번 삭제 시간관련데이터 앞으로 이동 
        for idx, col in enumerate(["년도","월","주차","공휴일"]):
            mlTable_dict[key] =Service.reorder_columns(col_name=col,df=mlTable_dict[key] ,target_idx=idx)
        mlTable_dict[key]['주중주말'] = ['SAT' if day in [5, 6] else 'DAY' for day in mlTable_dict[key]['요일']]
        mlTable_dict[key]=Service.reorder_columns(col_name='주중주말',df= mlTable_dict[key],target_idx=4)

    # 시간대 칼럼 이름 변경 ~시 인원으로 
    print(Service.colored_text("시간대 칼럼 이름 변경 ~시 인원으로",'yellow'))
    for key in  mlTable_dict.keys():
        colnamelist =mlTable_dict[key].columns.tolist()
        시간대시작index=Service.indexFind(colnamelist,'06')[0]
        # print(*colnamelist[시간대시작index:])
        for colname in colnamelist[시간대시작index:]:
            # print(f'{colname[:2]}시인원')
            if '이전' in colname or '06시이전'==colname:
                mlTable_dict[key].rename({colname:f'05시인원'}, inplace=True, axis=1)    
            elif '합' in colname:
                # 합 인원 칼럼 삭제
                print()
                mlTable_dict[key].drop(colname,inplace=True,axis=1)
            elif colname[:2]=='00':
                mlTable_dict[key].rename({colname:f'24시인원'}, inplace=True, axis=1)
            else:
                mlTable_dict[key].rename({colname:f'{colname[:2]}시인원'}, inplace=True, axis=1)
    print(Service.colored_text("########### Feature Engineering 완료 #################",'yellow'))
    integrated_mltable_line_dict ={}
    # 배차시간표는 Line 별로되어있기때문에 나눠서 머신러닝 만듬... 
    for key in mlTable_dict.keys():

        data_name=key # 'subway19'
        print(Service.colored_text(f"#___ data name = {data_name}",'yellow'))
        savefilename_indexName = data_name.split('subway')[-1]
        mlTable=mlTable_dict[key]
        
        mlTable_line_dict  ={}
        for line in range(1,9):
            # print(mlTable[mlTable['호선']==line])
            try: 
                mlt= mlTable[mlTable['호선']==line]
                # print(mlt.columns)
                line_배치 = pd.read_csv(f'../Data/지하철배차시간데이터/StationInfo_{savefilename_indexName}_{i}_호선배차.csv')
                mlTable_line_dict[f'{key}_{line}호선']= pd.merge(line_배치,mlt,on = ['역사코드','주중주말','호선'])
                if len(mlTable_line_dict[f'{key}_{line}호선']): 
                    print("아무것도 없습니다!!!!!")
                    print("아무것도 없습니다!!!!!")
                    print("아무것도 없습니다!!!!!")
                    print("아무것도 없습니다!!!!!")
                    print("그이유는!!!!!!")
                    print(mlt.columns)
                    print("mlt row 개수",len(mlt))
                    print(line_배치.columns)
                    print('line배치 row 개수 :',len(line_배치))
                    print('주중주말호선역사코드 가 제대로 되어있나확인해보세요')
                    print("역사코드",np.array(mlt['역사코드']).dtype)
                    print("호선",np.array(mlt['호선']).dtype)
                    print("주중주말",np.array(mlt['주중주말']).dtype)
                    
                    mlTable_line_dict[f'{key}_{line}호선']= pd.merge(line_배치,mlt,on = ['역사코드','주중주말'])
                    if len(mlTable_line_dict[f'{key}_{line}호선']): 
                        error_table[f'{key}_{line}호선']=[mlt,line_배치]

            except: continue
        # integrated_mltable_line_dict[f'{key}_line_dict']=mlTable_line_dict


        mlTable_dict[key]=mlTable_line_dict
    print(Service.colored_text("########### 호선별 mlTable 생성 완료 #################",'yellow'))
    # return integrated_mltable_line_dict
    # return mlTable_dict
    ## 배차 시간대 칼럼 이름 ~배차로 바꾸기 
    for key in mlTable_dict.keys():
        for line in mlTable_dict[key].keys():
            table = mlTable_dict[key][line]
            colnamelist = table.columns
            배차StartIndex = Service.indexFind(colnamelist=colnamelist.tolist(),search_target_word='05')[0]
            배차FinalIndex = Service.indexFind(colnamelist=colnamelist.tolist(),search_target_word='24')[0]
            for colname in colnamelist[배차StartIndex:배차FinalIndex+1]:
                # print(f'{colname[:2]}시인원')
                table.rename({colname:f'{colname[:2]}배차'}, inplace=True, axis=1)
            mlTable_dict[key][line]=table
            print(mlTable_dict[key][line])
    print(Service.colored_text("###########배차 시간대 칼럼 이름 ~배차로 바꾸기 완료 #################",'yellow'))
    # print("#"*20)   
    # print(mlTable_dict)
    np.save(f'../Data/mlTables/mlTable_dict_{save_mlTable_dict_fileName}.npy',mlTable_dict, allow_pickle=True)
    return error_table,mlTable


In [4]:
## subwaydict npy load 
#Data dictionalry path 
mlt_integrate_dict = {}
error_integrate_dict = {}
data_dictionary_path ='../Data/DataDictionary'
for file in os.listdir(data_dictionary_path):
    if file.split(".")[-1]=="npy":
        # print(file)
        savename = file.split(".npy")[0].split("_")[-1]
        print(savename)
        subway_dict = np.load(os.path.join(data_dictionary_path,file),allow_pickle=True).tolist()
        error_table,mlTable=mlTableGen(subway_dict,save_mlTable_dict_fileName=savename)
        error_integrate_dict[f'{savename}']=error_table
        mlt_integrate_dict[f'{savename}']=mlTable

22to23
___ 🟡 ML_TABLE_GENERATOR. 
______ 📌 머신러닝을 위한 Feature table 생성, 날짜별 승하차정보와 배차정보, 날짜적 특징을 보유한 테이블 생성
🔹 0 key : subway23_0
___ 🟡 DATA_PREPROCESSING_TOANALYSIS. 
______ 📌 데이터 통합 정제 함수!!!
columns ---👇
['연번', '수송일자', '호선', '역번호', '역명', '승하차구분', '06시이전', '06-07시간대', '07-08시간대', '08-09시간대', '09-10시간대', '10-11시간대', '11-12시간대', '12-13시간대', '13-14시간대', '14-15시간대', '15-16시간대', '16-17시간대', '17-18시간대', '18-19시간대', '19-20시간대', '20-21시간대', '21-22시간대', '22-23시간대', '23-24시간대', '24시이후']
 1. 연번을 삭제합니다. 
 2."역명" ->"역이름", "역번호"->"역사코드 ".
 3. 호선 데이터가 object 입니다. 
   - 1호선
   - 2호선
   - 3호선
 ..
 😀호선을 integer 로 만듭니다.
 3-1. 호선 을 제거한 이름 unique : , 1, 2, 3, 4, 5, 6, 7, 8
 ✅변경된 호선 칼럼의 data type : int64
___ 🟡 DATAINFOPROCESSING. 
______ 📌 Data frame 의 정제해야할 부분을 체크해주는 함수 입니다
  1️⃣ Data row/colum numbers : 98603/25
  2️⃣ null ceck 결과Null 없는 clean data!
  3️⃣ Column  Information (중복체크)
	idx.columName |				 |Colum Info(dtype)|** 
	 ----------------------------------------------------------------
	0.[수송일자(object)

In [5]:
# ## subwaydict npy load 
# #Data dictionalry path 
# mlt_integrate_dict = {}
# data_dictionary_path ='../Data/DataDictionary'
# for file in os.listdir(data_dictionary_path):
#     if file.split(".")[-1]=="npy":
#         # print(file)
#         savename = file.split(".npy")[0].split("_")[-1]
#         print(savename)
#         subway_dict = np.load(os.path.join(data_dictionary_path,file),allow_pickle=True).tolist()
#         mlTable_dict=mlTableGen(subway_dict,save_mlTable_dict_fileName=savename)
#         mlt_integrate_dict[f'{savename}']=mlTable_dict


#mlTable integration 결과 플랏 및 저장 
# 

        # print(mlt_integrate_dict[section][year])
np.save('../Data2/mlTables/integratedMLTable_dict.npy',mlt_integrate_dict,allow_pickle=True)

In [6]:
table,batch=error_integrate_dict['22to23']['subway23_0_3호선']  
# table.iloc[:,:10]
# mlTable_line_dict[f'{key}_{line}호선']= pd.merge(line_배치,mlt,on = ['역사코드','주중주말'])
pd.merge(table,batch,on = ['역사코드','주중주말','호선'])

,년도,월,주차,공휴일,주중주말,수송일자,호선,역사코드,역이름,요일,...,15,16,17,18,19,20,21,22,23,24
0,2022,1,52,0,DAY,2023-01-01,3,309,지축,0,...,13.0,13.0,15.0,14.0,14.0,13.0,9.0,11.0,4.0,0.0
1,2022,1,52,0,DAY,2023-01-01,3,309,지축,0,...,13.0,13.0,15.0,14.0,14.0,13.0,9.0,11.0,4.0,0.0
2,2023,1,1,1,DAY,2023-01-02,3,309,지축,1,...,13.0,13.0,15.0,14.0,14.0,13.0,9.0,11.0,4.0,0.0
3,2023,1,1,1,DAY,2023-01-02,3,309,지축,1,...,13.0,13.0,15.0,14.0,14.0,13.0,9.0,11.0,4.0,0.0
4,2023,1,1,1,DAY,2023-01-03,3,309,지축,2,...,13.0,13.0,15.0,14.0,14.0,13.0,9.0,11.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12020,2023,6,24,1,SAT,2023-06-17,3,321,충무로,6,...,21.0,20.0,20.0,19.0,20.0,18.0,18.0,16.0,12.0,0.0
12021,2023,6,25,1,SAT,2023-06-23,3,321,충무로,5,...,21.0,20.0,20.0,19.0,20.0,18.0,18.0,16.0,12.0,0.0
12022,2023,6,25,1,SAT,2023-06-23,3,321,충무로,5,...,21.0,20.0,20.0,19.0,20.0,18.0,18.0,16.0,12.0,0.0
12023,2023,6,26,1,SAT,2023-06-30,3,321,충무로,5,...,21.0,20.0,20.0,19.0,20.0,18.0,18.0,16.0,12.0,0.0


In [7]:
for i in error_integrate_dict.keys():
    print(i)
    print(error_integrate_dict[i].keys())
    for j in error_integrate_dict[i].keys():
        print(j)
        table1,table2=error_integrate_dict[i][j]
        print(table1.head(3))
        print(table2.head(3))
error_integrate_dict['22to23']['subway23_0_3호선']  

22to23
dict_keys(['subway23_0_3호선', 'subway23_1_3호선', 'subway23_2_3호선'])
subway23_0_3호선
       년도  월  주차  공휴일 주중주말        수송일자  호선  역사코드  역이름  요일  ... 15시인원  16시인원  \
120  2022  1  52    0  DAY  2023-01-01   3   309   지축   0  ...   207    184   
121  2022  1  52    0  DAY  2023-01-01   3   309   지축   0  ...   204    216   
122  2022  1  52    0  DAY  2023-01-01   3   310  구파발   0  ...   878    939   

     17시인원  18시인원  19시인원  20시인원  21시인원  22시인원  23시인원  24시인원  
120    160    139    128    123     75     44     20      0  
121    224    203    168    167    135    119     76      0  
122    774    631    449    449    354    280     49      1  

[3 rows x 31 columns]
   역사코드  호선 주중주말   05    06    07    08    09    10    11  ...    15    16  \
0   309   3  DAY  3.0  12.0  16.0  19.0  15.0  13.0  14.0  ...  13.0  13.0   
1   309   3  SAT  2.0   8.0  12.0  13.0  11.0  11.0  14.0  ...  11.0  10.0   
2   310   3  DAY  2.0  14.0  22.0  23.0  22.0  19.0  14.0  ...  13.0  14.0   

     17    

[         년도  월  주차  공휴일 주중주말        수송일자  호선  역사코드   역이름  요일  ... 15시인원  \
 120    2022  1  52    0  DAY  2023-01-01   3   309    지축   0  ...   207   
 121    2022  1  52    0  DAY  2023-01-01   3   309    지축   0  ...   204   
 122    2022  1  52    0  DAY  2023-01-01   3   310   구파발   0  ...   878   
 123    2022  1  52    0  DAY  2023-01-01   3   310   구파발   0  ...   721   
 124    2022  1  52    0  DAY  2023-01-01   3   311   연신내   0  ...  1353   
 ...     ... ..  ..  ...  ...         ...  ..   ...   ...  ..  ...   ...   
 98240  2023  6  26    1  SAT  2023-06-30   3   340  가락시장   5  ...   520   
 98241  2023  6  26    1  SAT  2023-06-30   3   341  경찰병원   5  ...   389   
 98242  2023  6  26    1  SAT  2023-06-30   3   341  경찰병원   5  ...   319   
 98243  2023  6  26    1  SAT  2023-06-30   3   342    오금   5  ...   290   
 98244  2023  6  26    1  SAT  2023-06-30   3   342    오금   5  ...   341   
 
        16시인원  17시인원  18시인원  19시인원  20시인원  21시인원  22시인원  23시인원  24시인원  
 120      184 

In [8]:

# mlTable_dict['subway23_0'].columns
# for i in line_배치_dict.keys():
#     print(line_배치_dict[i].columns)

# # 배차시간표는 Line 별로되어있기때문에 나눠서 머신러닝 만듬... 
# for key in mlTable_dict.keys():
#     mlTable=mlTable_dict[key]
    
#     mlTable_line_dict  ={}
#     for line in range(1,9):
#         print(mlTable[mlTable['호선']==line])
#         try: 
#             mlt= mlTable[mlTable['호선']==line]
#             print(mlt.columns)
#             mlTable_line_dict[f'{key}_{line}호선']= pd.merge(line_배치_dict[f'{line}호선'],mlt,on = ['역사코드','주중주말','호선','역이름'])
#         except: pass
#     mlTable_dict[key]=mlTable_line_dict
# mlTable_dict['subway23_0']
# ## 배차 시간대 칼럼 이름 ~배차로 바꾸기 

# for key in mlTable_dict.keys():
#     for line in mlTable_dict[key].keys():
#         table = mlTable_dict[key][line]
#         colnamelist = table.columns
#         배차StartIndex = Service.indexFind(colnamelist=colnamelist.tolist(),search_target_word='05')[0]
#         배차FinalIndex = Service.indexFind(colnamelist=colnamelist.tolist(),search_target_word='24')[0]
#         for colname in colnamelist[배차StartIndex:배차FinalIndex+1]:
#             # print(f'{colname[:2]}시인원')
#             table.rename({colname:f'{colname[:2]}배차'}, inplace=True, axis=1)
#         mlTable_dict[key][line]=table

# mlTable_dict.keys()
# # mlTable_dict['subway23_0']['subway23_0_1'].columns
# # 칼럼 순서이동 및 쓸데없는 칼럼 버리기 
# move_target_colname_list =[ '년도', '월', '주차', '공휴일', '수송일자', '요일', '승하차구분']

# # for line in mlTable_dict['subway23_0']['subway23_0_1']:
    
# for key in mlTable_dict.keys():
#     for line in mlTable_dict[key].keys():
#         table = mlTable_dict[key][line]
#         table.drop('주중주말', axis=1, inplace=True)
#         table.drop('수송일자', axis=1, inplace=True)
#         table.drop('년도', axis=1, inplace=True)
#         mlTable_dict[key][line]=table

# mlTable_dict['subway23_0']['subway23_0_1호선']['승하차구분']#.columns#.iloc[:3,:23].head()
# mlTable_dict['subway23_0']['subway23_0_1호선'].columns


In [2]:
# mlTable check
integrated_MLT=np.load('../Data/mlTables/integratedMLTable_dict.npy',allow_pickle=True).tolist()


In [3]:
# Line 별 통합 feature table 만들기!!!! 
## 7호선만 해보자..
line_1_dict={}
line_2_dict={}
line_3_dict={}
line_4_dict={}
line_5_dict={}
line_6_dict={}
line_7_dict={}
line_8_dict={}
integrated_MLT
for section in integrated_MLT.keys():
    print("section: ", section)
    for year in integrated_MLT[section]:
        print(" |-year :",year)
        for name in integrated_MLT[section][year].keys():
            print("   |-",name)
            match name:
                case name if '1호선' in name : 
                    line_1_dict[f'{name}'] = integrated_MLT[section][year][name]
                case name if '2호선' in name : 
                    line_2_dict[f'{name}'] = integrated_MLT[section][year][name]
                case name if '3호선' in name : 
                    line_3_dict[f'{name}'] = integrated_MLT[section][year][name]
                case name if '4호선' in name : 
                    line_4_dict[f'{name}'] = integrated_MLT[section][year][name]
                case name if '5호선' in name : 
                    line_5_dict[f'{name}'] = integrated_MLT[section][year][name]
                case name if '6호선' in name : 
                    line_6_dict[f'{name}'] = integrated_MLT[section][year][name]
                case name if '7호선' in name : 
                    line_7_dict[f'{name}'] = integrated_MLT[section][year][name]
                case name if '8호선' in name : 
                    line_8_dict[f'{name}'] = integrated_MLT[section][year][name]

line_1_dict.keys()
line_array=[
    line_1_dict,
    line_2_dict,
    line_3_dict,
    line_4_dict,
    line_5_dict,
    line_6_dict,
    line_7_dict,
    line_8_dict
    ]

section:  22to23
 |-year : subway23_0
   |- subway23_0_1호선
   |- subway23_0_2호선
   |- subway23_0_3호선
   |- subway23_0_4호선
   |- subway23_0_5호선
   |- subway23_0_6호선
   |- subway23_0_7호선
   |- subway23_0_8호선
 |-year : subway23_1
   |- subway23_1_1호선
   |- subway23_1_2호선
   |- subway23_1_3호선
   |- subway23_1_4호선
   |- subway23_1_5호선
   |- subway23_1_6호선
   |- subway23_1_7호선
   |- subway23_1_8호선
 |-year : subway23_2
   |- subway23_2_1호선
   |- subway23_2_2호선
   |- subway23_2_3호선
   |- subway23_2_4호선
   |- subway23_2_5호선
   |- subway23_2_6호선
   |- subway23_2_7호선
   |- subway23_2_8호선
 |-year : subway22
section:  20to21
 |-year : subway21
   |- subway21_1호선
   |- subway21_2호선
   |- subway21_3호선
   |- subway21_4호선
   |- subway21_5호선
   |- subway21_6호선
   |- subway21_7호선
   |- subway21_8호선
 |-year : subway20
   |- subway20_1호선
   |- subway20_2호선
   |- subway20_3호선
   |- subway20_4호선
   |- subway20_5호선
   |- subway20_6호선
   |- subway20_7호선
   |- subway20_8호선
section:  18to19
 |-year : subway19
  

In [ ]:
len(line_5_dict['subway23_0_5호선'])

0

In [ ]:
line_name=list(line_1_dict.keys())[0].split('_')[-1]
year

'subway18'

In [ ]:
def merge_line_table(line_dict):
    import pandas as pd
    line_name=list(line_dict.keys())[0].split('_')[-1]
    # line_1_dict에 저장된 데이터프레임을 합칠 리스트 초기화
    dataframes = []
    # line_1_dict의 각 데이터프레임을 순회하며 빈 데이터프레임이 아닌 경우만 리스트에 추가

    for i in line_dict.keys():
            if not line_dict[i].empty:  # 데이터프레임이 비어있지 않은지 확인
                if '24시인원' not in line_dict[i].columns:
                    line_dict[i]['24시인원']=0
                # 인덱스를 재설정하여 고유한 인덱스를 가집니다.
                print(i, len(line_dict[i].columns), sep="\n")
                dataframes.append(line_dict[i])
    # 리스트에 있는 모든 데이터프레임을 행 방향으로 합침
    if dataframes:
        merged_table_line = pd.concat(dataframes, ignore_index=True)
        save_folder = '../Data/mlTables/integrated_mlt_line'
        os.makedirs(save_folder, exist_ok=True)  # 저장 폴더가 없으면 생성
        # 결과 출력
        print(merged_table_line)
        merged_table_line.to_csv(os.path.join(save_folder,f"{line_name}_통합_mlTable.csv"))

for i in line_array:
    merge_line_table(i)


subway21_1호선
52
subway20_1호선
52
subway19_1호선
52
subway18_1호선
52
       역사코드  호선_x 주중주말  05배차  06배차  07배차  08배차  09배차  10배차  11배차  ...  15시인원  \
0       150     1  DAY   5.5  13.5  16.5  18.0  15.5  13.5  16.5  ...   1117   
1       150     1  DAY   5.5  13.5  16.5  18.0  15.5  13.5  16.5  ...   1022   
2       150     1  DAY   5.5  13.5  16.5  18.0  15.5  13.5  16.5  ...   1714   
3       150     1  DAY   5.5  13.5  16.5  18.0  15.5  13.5  16.5  ...   1151   
4       150     1  DAY   5.5  13.5  16.5  18.0  15.5  13.5  16.5  ...   1529   
...     ...   ...  ...   ...   ...   ...   ...   ...   ...   ...  ...    ...   
29215   159     1  SAT   4.5   9.5  12.0  12.5  12.5  12.5  14.5  ...   1793   
29216   159     1  SAT   4.5   9.5  12.0  12.5  12.5  12.5  14.5  ...    820   
29217   159     1  SAT   4.5   9.5  12.0  12.5  12.5  12.5  14.5  ...    662   
29218   159     1  SAT   4.5   9.5  12.0  12.5  12.5  12.5  14.5  ...   1625   
29219   159     1  SAT   4.5   9.5  12.0  12.5  12.5  12

In [4]:
mlTable_dict


NameError: name 'mlTable_dict' is not defined

---
### ml prediction 
- 머신러닝을 위한 feature table  불러와서 training, test table 을 저장함. 
- 승하차로 나누어서 저장함. 


In [ ]:
# test = mlTable_dict['subway23_0']['subway23_0_1호선'].columns

def MLPredciton_ver02(featureTable,saveFileName) :
        """
        ##### 📌 Description : train,target이 모두있는 feature table 을 입력받아 ML model 별 score 출력후 모델 반환
        ##### 📌 Date : 2024.06.13
        ##### 📌 Author : pdg
        ##### 📌 Detail: 승하차구분없고, in,out 구분이 없다. 
            🔸 Returns: - 
        ##### 📌 Updata:
            🟦 2024.06.13 by pdg : 모델 함수 만듬. 
        """
        import pandas as pd, numpy as np
        import matplotlib.pyplot as plt 
        from sklearn.model_selection import train_test_split
        from sklearn.multioutput import MultiOutputRegressor
        from sklearn.neighbors import KNeighborsRegressor

        ## feature table input column
        input_column=['월', '주차', '공휴일', '요일', '역사코드', '주중', '주말',
            'latitude', 'longitude', '05배차', '06배차', '07배차', '08배차', '09배차', '10배차',
            '11배차', '12배차', '13배차', '14배차', '15배차', '16배차', '17배차', '18배차', '19배차',
            '20배차', '21배차', '22배차', '23배차', '24배차']
        ## featrue table output(target) column -> multi output 
        target_column=['05시인원', '06시인원', '07시인원',
        '08시인원', '09시인원', '10시인원', '11시인원', '12시인원', '13시인원', '14시인원', '15시인원',
        '16시인원', '17시인원', '18시인원', '19시인원', '20시인원', '21시인원', '22시인원', '23시인원',
        '24시인원']
        ## 승하차  구분 
        mlTable_승차 = featureTable[featureTable['승하차구분']=='승차']
        mlTable_하차 = featureTable[featureTable['승하차구분']=='하차']
        ## 승차 input, target
        mlTable_승차_input = mlTable_승차[input_column]
        mlTable_승차_target =mlTable_승차[target_column]
        
        ## 하차 input, target 
        mlTable_하차_input= mlTable_하차[input_column]
        mlTable_하차_target= mlTable_하차[target_column]
        
        ## training test spilit (승차)
        train_input_승차, test_input_승차, train_target_승차, test_target_승차 = \
            train_test_split(mlTable_승차_input,
                            mlTable_승차_target, 
                            test_size=0.2,
                            # random_state=42
                            )
            
        ## training test split (하차)
        train_input_하차, test_input_하차, train_target_하차, test_target_하차 = \
            train_test_split(mlTable_하차_input,
                            mlTable_하차_target, 
                            test_size=0.2,
                            # random_state=42
                            )
        ## trainin test _dict  저장 
        ml_train_test_dict ={
            'train_input_승차':train_input_승차,
            'test_input_승차':test_input_승차,
            'train_target_승차':train_target_승차,
            'test_target_승차':test_target_승차,
            'train_input_하차':train_input_하차,
            'test_input_하차':test_input_하차,
            'train_target_하차':train_target_하차,
            'test_target_하차':test_target_하차
        }
        np.save(f'../MLModels/TrainingTargetDataSet_dictionary/{saveFileName}.npy',ml_train_test_dict)
        # return ml_train_test_dict

for key in mlTable_dict.keys():
    for line in mlTable_dict[key].keys():
        featureTable=mlTable_dict[key][line]
        MLPredciton_ver02(featureTable=featureTable, saveFileName=f'{line}_data_dict')


KeyError: '승하차구분'

In [ ]:
data_dict=np.load('../MLModels/TrainingTargetDataSet_dictionary/subway23_0_1호선_data_dict.npy',allow_pickle=True).tolist()
def regression_model(data_dict,승하차,saveFileName):
    import pandas as pd, numpy as np
    import matplotlib.pyplot as plt 
    from sklearn.model_selection import train_test_split
    from sklearn.multioutput import MultiOutputRegressor
    from sklearn.neighbors import KNeighborsRegressor
    knn_regressor = KNeighborsRegressor(n_neighbors=3)
    ## Multi Output Setting
    multi_output_regressor = MultiOutputRegressor(knn_regressor)
    multi_output_regressor.fit(data_dict[f'train_input_{승하차}'], data_dict[f'train_target_{승하차}'])
    score = multi_output_regressor.score(data_dict[f'test_input_{승하차}'], data_dict[f'test_target_{승하차}'])
    predictions= multi_output_regressor.predict(data_dict[f'test_input_{승하차}'])
    print(Service.colored_text(f'{saveFileName}_{승하차}예측--','yellow'))
    print(Service.colored_text(f'✅Model score({승하차}): {score}','red'))
    print("주차     요일 시간대별 예측 :",*[f"{i}시" for i in range(5,25)], sep='\t')
    for idx,시간대별예측 in enumerate(predictions):
        if idx <3:
            주차 = data_dict[f'test_input_{승하차}'].to_numpy()[idx][1]
            요일 = data_dict[f'test_input_{승하차}'].to_numpy()[idx][3]
            실제치 = data_dict[f'test_target_{승하차}'].to_numpy()[idx]
            match 요일:
                case 요일 if 요일 == 0: 요일_str = '일'; 
                case 요일 if 요일 == 1: 요일_str = '월'; 
                case 요일 if 요일 == 2: 요일_str = '화'; 
                case 요일 if 요일 == 3: 요일_str = '수'; 
                case 요일 if 요일 == 4: 요일_str = '목'; 
                case 요일 if 요일 == 5: 요일_str = '금'; 
                case 요일 if 요일 == 6: 요일_str = '토'; 
                case _:print()
            print(f"{주차}주차 {요일_str}요일 시간대별 예측 :", *list(map(int,(시간대별예측))), sep='\t')
            print(f"{주차}주차 {요일_str}요일 시간대별 실제 :", *실제치, sep='\t')
            print("---"*200)
    import joblib ## model 저장 용 함수 
    filename = f'../MLModels/knn_regressor_{saveFileName}_{승하차}.h5'
    print(f"😀😀😀😀{filename} 을 저장합니다 😀😀😀")
    joblib.dump(multi_output_regressor, filename)
        #return multi_output_regressor
# regression_model(data_dict,'승차', saveFileName=)
# regression_model(data_dict,'하차')


In [ ]:
for i in os.listdir('../MLModels/TrainingTargetDataSet_dictionary'):
    print(i)
    data_dict=np.load(os.path.join('../MLModels/TrainingTargetDataSet_dictionary/',i),allow_pickle=True).tolist()
    regression_model(data_dict,'승차', saveFileName=i.split('_data_dict.npy')[0])
    regression_model(data_dict,'하차', saveFileName=i.split('_data_dict.npy')[0])
    

subway23_0_8호선_data_dict.npy
subway23_0_8호선_승차예측--
✅Model score(승차): 0.8192247184144341
주차     요일 시간대별 예측 :	5시	6시	7시	8시	9시	10시	11시	12시	13시	14시	15시	16시	17시	18시	19시	20시	21시	22시	23시	24시
2주차 목요일 시간대별 예측 :	412	831	2091	2729	1271	689	580	609	582	508	474	535	748	862	354	233	180	118	52	13
2주차 목요일 시간대별 실제 :	416	850	2161	2706	1279	688	611	572	597	538	466	567	768	891	350	222	144	137	49	18
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
24주차 토요일 시간대별 예측 :

23년은 여러개로쪼개져있으니 예측률이 다양해도 맞추는게 의미가 없다.
빨리 22,이랑 다 합친걸로 하는걸 하도록 하자. 